# Import

In [1]:
import pandas as pd
import numpy as np

from astropy.io import fits
from forecast_SN_GW import get_hubblefit

# Load data

In [2]:
data_path = '../'

In [3]:
d_ps1 = pd.read_csv(data_path+'Data/SN_ps_snls_sdss_hst.csv', sep=' ', index_col='CID')
ps1_pd = d_ps1[d_ps1['IDSURVEY'] == 15]

d_X = pd.read_csv(data_path+'Data/jla_lcparams.txt', sep=' ', index_col='#name')

bias_data = fits.getdata(data_path+'Data/covmat/C_bias.fits')
cal_data = fits.getdata(data_path+'Data/covmat/C_cal.fits')
dust_data = fits.getdata(data_path+'Data/covmat/C_dust.fits')
host_data = fits.getdata(data_path+'Data/covmat/C_host.fits')
model_data = fits.getdata(data_path+'Data/covmat/C_model.fits')
nonia_data = fits.getdata(data_path+'Data/covmat/C_nonia.fits')
pecvel_data = fits.getdata(data_path+'Data/covmat/C_pecvel.fits')
stat_data = fits.getdata(data_path+'Data/covmat/C_stat.fits')
covX = bias_data + cal_data + dust_data + host_data + model_data + nonia_data + pecvel_data + stat_data

# Reproduce JLA results

## Build data array for JLA fit

In [4]:
mass = np.zeros_like(d_X['3rdvar'].values)
for i, m in enumerate(d_X['3rdvar'].values):
    if m > 10:
        mass[i] = -1.
X = np.array([d_X['mb'].values, d_X['x1'].values, d_X['color'].values, mass])

d_sigma = pd.read_csv(data_path+'Data/covmat/sigma_mu.txt', sep=' ')
S = np.array([d_sigma['#sigma_coh'].values, d_sigma['#sigma_lens'].values,
              d_sigma['#z'].values])
S = S.T

## JLA fit

In [5]:
g = get_hubblefit(X.T, covX, d_X['zcmb'].values, d_X['zcmb'].values, S[:, 0], S[:, 1])

In [6]:
g.fit()
g.minuit.values

FCN = 682.891139136 
 TOTAL NCALL = 160 
 NCALLS = 160 
 
 
 EDM = 4.31626452499e-06 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,Mb,-19.0485,0.0231721,0,0,,,
2,alpha1,0.141227,0.00658992,0,0,,,
3,alpha2,-3.1013,0.0807218,0,0,,,
4,alpha3,-0.069987,0.0231102,0,0,,,
5,omgM,0.295323,0.0339014,0,0,0.0,1.0,


{'Mb': -19.04845558418629,
 'alpha1': 0.14122692809980045,
 'alpha2': -3.101297018846571,
 'alpha3': -0.06998699746790762,
 'omgM': 0.2953229798598163}

In [7]:
g.minuit.errors

{'Mb': 0.02317210816121692,
 'alpha1': 0.006589922760960733,
 'alpha2': 0.08072184422519378,
 'alpha3': 0.023110224151747582,
 'omgM': 0.0339013771581024}

# Fit with linear correction in redshift 

## Build data array 

In [8]:
X = np.array([d_X['mb'].values, d_X['x1'].values, d_X['color'].values, d_X['zcmb'].values])

d_sigma = pd.read_csv(data_path+'Data/covmat/sigma_mu.txt', sep=' ')
S = np.array([d_sigma['#sigma_coh'].values, d_sigma['#sigma_lens'].values,
              d_sigma['#z'].values])
S = S.T

## Fit 

In [9]:
g = get_hubblefit(X.T, covX, d_X['zcmb'].values, d_X['zcmb'].values, S[:, 0], S[:, 1])
g.fit()

FCN = 691.236079145 
 TOTAL NCALL = 203 
 NCALLS = 203 
 
 
 EDM = 7.35165110526e-07 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,Mb,-19.0682,0.0232206,0,0,,,
2,alpha1,0.136241,0.00637906,0,0,,,
3,alpha2,-3.10156,0.080593,0,0,,,
4,alpha3,-0.223353,0.193771,0,0,,,
5,omgM,0.470514,0.171655,0,0,0.0,1.0,


(0.15356858018842226, 0.005000062920239221)

In [10]:
g.minuit.values

{'Mb': -19.06819050370749,
 'alpha1': 0.13624144563029492,
 'alpha2': -3.1015613996575304,
 'alpha3': -0.22335319735181183,
 'omgM': 0.4705137354352047}

In [11]:
g.minuit.errors

{'Mb': 0.02322060016959228,
 'alpha1': 0.006379063409005724,
 'alpha2': 0.08059299566015492,
 'alpha3': 0.1937706827490381,
 'omgM': 0.17165514492437767}

# Combine with standard sirens

In [12]:
mu_sirens = np.genfromtxt('../GWMock/mock_catalogue.txt')
X_combined = np.zeros((len(X.T)+len(mu_sirens),len(X.T[0])))

In [13]:
X_combined[:-len(X.T),0] = mu_sirens[:,1]
X_combined[len(mu_sirens):,:] = X.T[:,:]
print X_combined

[[ 4.67820264e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 4.35708662e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 4.45082341e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 ...
 [ 1.58955010e+01  6.18766000e-01 -5.54110000e-02  2.38100000e-02]
 [ 1.60682680e+01  7.60605000e-01  5.21860000e-02  2.38670000e-02]
 [ 1.57185400e+01  4.30639000e-01 -3.83670000e-02  2.20680000e-02]]


In [14]:
cov_combined = np.zeros(((len(X.T)+len(mu_sirens))*3, (len(X.T)+len(mu_sirens))*3))

In [15]:
np.shape(covX)

(2220, 2220)

In [16]:
for i, err in enumerate(mu_sirens[:,2]):
    cov_combined[i*3,i*3] = err**2
cov_combined[len(mu_sirens)*3:,len(mu_sirens)*3:] = covX
sirens = np.zeros(len(X.T)+len(mu_sirens))
sirens[len(mu_sirens):] = sirens[len(mu_sirens):]+1

In [17]:
z = np.concatenate((mu_sirens[:,0], d_X['zcmb'].values), axis=None)
sig_coh = np.concatenate((np.zeros(len(mu_sirens)), S[:, 0]), axis=None)
sig_lens = np.concatenate((np.zeros(len(mu_sirens)), S[:, 1]), axis=None)

In [ ]:
g = get_hubblefit(X_combined, cov_combined, z, z, sig_coh, sig_lens, sirens=sirens)
g.fit()
g.minuit.values
g.minuit.errors

In [ ]:
from matplotlib import pyplot as plt
from forecast_SN_GW import distance_modulus_th
plt.scatter(d_X['zcmb'].values, X.T[:,0]+19.05, c='r')
plt.scatter(mu_sirens[:,0], mu_sirens[:,1], c='b')
z = np.linspace(0.,5,100)
plt.plot(z, distance_modulus_th(0.3,z,z), c='black')
# plt.xlim(-0.0,1)
plt.show()

In [ ]:
g.minuit.fval/g.

In [ ]:
g.chi2_per_dof

In [ ]:
g.dof